In [6]:
%load_ext autoreload
%autoreload 2

from artfinder import CrossrefArticle, Crossref, load_csv
import pandas as pd
import datetime
import re
import logging
from json import dumps

logging.basicConfig(level=logging.INFO)
crosref = Crossref(app='artfinder', email='aapopov1@mephi.ru')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
df = load_csv('database/processed/kabashin_full.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   publisher               251 non-null    string        
 1   license                 149 non-null    object        
 2   is_referenced_by_count  251 non-null    int64         
 3   link                    203 non-null    object        
 4   authors                 251 non-null    object        
 5   abstract                78 non-null     string        
 6   title                   251 non-null    string        
 7   doi                     251 non-null    string        
 8   type                    251 non-null    string        
 9   journal                 247 non-null    string        
 10  issn                    210 non-null    string        
 11  volume                  206 non-null    string        
 12  issue                   169 non-null    string    

In [11]:
all_refs = []
for ret_list in df['references']:
    if ret_list is not None:
        all_refs.extend(ret_list)

print(f"Total references: {len(all_refs)}")
all_refs = list(set(all_refs))
print(f"Unique references: {len(all_refs)}")
all_refs[:10]

Total references: 7595
Unique references: 4181


['10.1063/1.1628830',
 '10.1021/ac8007219',
 '10.1021/ph5002186',
 '10.1016/j.jcrysgro.2004.04.120',
 '10.1038/s41563-022-01287-1',
 '10.1021/jp002340c',
 '10.2967/jnumed.112.107540',
 '10.1016/S0304-3835(99)00105-6',
 '10.1070/RC1995v064n10ABEH000188',
 '10.1259/bjr/13169882']

In [12]:
results, failed = crosref.get_dois(all_refs[:100], 10)

In [14]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 0
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   publisher               100 non-null    string        
 1   license                 66 non-null     object        
 2   is_referenced_by_count  100 non-null    object        
 3   link                    97 non-null     object        
 4   authors                 100 non-null    object        
 5   abstract                20 non-null     string        
 6   title                   100 non-null    string        
 7   doi                     100 non-null    string        
 8   type                    100 non-null    string        
 9   journal                 100 non-null    string        
 10  issn                    100 non-null    string        
 11  volume                  99 non-null     string        
 12  issue                   92 non-null     string        
 1

In [18]:
failed

['10.1002/9781118354162', '10.1201/b17170']

In [7]:
test_article = crosref.doi('10.1023/A:1005781517624')

In [8]:
test_article

{'indexed': {'date-parts': [[2024, 9, 5]],
  'date-time': '2024-09-05T22:38:35Z',
  'timestamp': 1725575915236},
 'reference-count': 0,
 'publisher': 'Springer Science and Business Media LLC',
 'issue': '1/2',
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'short-container-title': [],
 'published-print': {'date-parts': [[1997]]},
 'DOI': '10.1023/a:1005781517624',
 'type': 'journal-article',
 'created': {'date-parts': [[2002, 12, 21]],
  'date-time': '2002-12-21T18:13:52Z',
  'timestamp': 1040494432000},
 'page': '105-115',
 'source': 'Crossref',
 'is-referenced-by-count': 120,
 'title': [],
 'prefix': '10.1007',
 'volume': '33',
 'author': [{'given': 'Yoshinobu',
   'family': 'Nakagawa',
   'sequence': 'first',
   'affiliation': []},
  {'given': 'Hiroshi',
   'family': 'Hatanaka†',
   'sequence': 'additional',
   'affiliation': []}],
 'member': '297',
 'container-title': ['Journal of Neuro-Oncology'],
 'original-title': [],
 'deposited': {'date-parts': [[2012, 12,

In [9]:
CrossrefArticle(test_article)

Make a search query and construct pandas dataframe with results
* Valid fields for query are in Crossref.FIELDS_QUERY
* Valid fields for filter are in Crossref.FILTER_VALIDATOR
* Specifying the same filter several times results in OR semantics, while specifying different filters results in AND semantics
* [RESP-API documentation](https://github.com/CrossRef/rest-api-doc?tab=readme-ov-file#queries)



In [ ]:

df = crosref.query(author='Barcikowski').filter(from_pub_date='1993', type=['proceedings-article', 'journal-article']).get_df()
df.info()

INFO:crossref.restful:GET request to https://api.crossref.org/works, data={'query.author': 'Barcikowski', 'filter': 'from-pub-date:1993,type:proceedings-article,type:journal-article', 'cursor': '*', 'rows': 100}, headers={'user-agent': 'artfinder/undefined (undefined; mailto:aapopov1@mephi.ru) BasedOn: CrossrefAPI/1.6.0'}


INFO:crossref.restful:Sleep for 0.02 seconds
INFO:crossref.restful:GET request to https://api.crossref.org/works, data={'query.author': 'Barcikowski', 'filter': 'from-pub-date:1993,type:proceedings-article,type:journal-article', 'cursor': 'DnF1ZXJ5VGhlbkZldGNoBgAAAAAHhflEFi10RWc3ay1xUzB1LXA4djZWbEdpQ1EAAAAAE3AWCBZWX2tkV3YyWVFVcV9hOFZHWGN4cURnAAAAADMpzKEWTUpsaGN0RGFRbS1yN0ZYWTJ3MG5pUQAAAAAysi22Fjc3VnRybUJuUTJlZzI5LUZWOGtVbWcAAAAAHznOwhZwMUUtd3VRM1RzU3RxcXpMUVg3MFZBAAAAADRhZWIWY2hqVHNHWVVRYkdqcWtJNUJXbkk4Zw==', 'rows': 100}, headers={'user-agent': 'artfinder/undefined (undefined; mailto:aapopov1@mephi.ru) BasedOn: CrossrefAPI/1.6.0'}
INFO:crossref.restful:Sleep for 0.02 seconds
INFO:crossref.restful:GET request to https://api.crossref.org/works, data={'query.author': 'Barcikowski', 'filter': 'from-pub-date:1993,type:proceedings-article,type:journal-article', 'cursor': 'DnF1ZXJ5VGhlbkZldGNoBgAAAAAHhflEFi10RWc3ay1xUzB1LXA4djZWbEdpQ1EAAAAAE3AWCBZWX2tkV3YyWVFVcV9hOFZHWGN4cURnAAAAADMpzKEWTUps

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   publisher               548 non-null    string        
 1   license                 348 non-null    object        
 2   is_referenced_by_count  548 non-null    object        
 3   link                    486 non-null    object        
 4   authors                 548 non-null    object        
 5   abstract                184 non-null    string        
 6   title                   548 non-null    string        
 7   doi                     548 non-null    string        
 8   type                    548 non-null    string        
 9   journal                 548 non-null    string        
 10  issn                    491 non-null    string        
 11  volume                  480 non-null    string        
 12  issue                   369 non-null    string    

In [52]:
df['type'].unique()

<StringArray>
['journal-article', 'proceedings-article']
Length: 2, dtype: string

Save for marking data

In [ ]:
path_for_processing = ''
df[['title', 'abstract', 'doi']].to_excel(path_for_processing, index=False)

Save data

In [55]:
path_to_save = 'database/barcikowski.csv'
df.to_csv(path_to_save, index=False)